In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [2]:
class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        super(conv_block, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, **kwargs)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x

In [3]:
#red --> reduction 
class Inception_block(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(Inception_block, self).__init__()
        # 1x1 convolution branch
        self.branch1 = conv_block(in_channels, ch1x1, kernel_size=1, stride=1, padding=0)
        
        # 1x1 convolution followed by 3x3 convolution branch
        self.branch2 = nn.Sequential(
            conv_block(in_channels, ch3x3red, kernel_size=1, stride=1, padding=0),
            conv_block(ch3x3red, ch3x3, kernel_size=3, stride=1, padding=1)
        )
        
        # 1x1 convolution followed by 5x5 convolution branch
        self.branch3 = nn.Sequential(
            conv_block(in_channels, ch5x5red, kernel_size=1, stride=1, padding=0),
            conv_block(ch5x5red, ch5x5, kernel_size=5, stride=1, padding=2)
        )
        
        # 3x3 max pooling followed by 1x1 convolution branch
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            conv_block(in_channels, pool_proj, kernel_size=1, stride=1, padding=0)
        )
    
    def forward(self, x):
        branch1_output = self.branch1(x)
        branch2_output = self.branch2(x)
        branch3_output = self.branch3(x)
        branch4_output = self.branch4(x)
        output = torch.cat([branch1_output, branch2_output, branch3_output, branch4_output], 1)
        return output

In [6]:
class GoogLeNet(nn.Module):
    def __init__(self , in_channels , num_classes):
        super(GoogLeNet ,self).__init__()
        
        self.conv1 = conv_block(in_channels , out_channels = 64 , kernel_size = (7,7)  ,stride = (2,2) ,padding  = (3,3))
        self.maxpool1 = nn.MaxPool2d(kernel_size = 3 , stride = 2 , padding =1)
        self.conv2 = conv_block(64 , 192 , kernel_size = 3  ,stride = 1 ,padding  = 1)
        self.maxpool2 = nn.MaxPool2d(kernel_size = 3 , stride = 2 , padding =1)

        self.Inception3a = Inception_block(192 ,64 ,96 ,128 ,16 ,32 ,32)
        self.Inception3b = Inception_block(256 ,128 ,128 ,192 ,32 ,96 ,64)
        self.maxpool3 = nn.MaxPool2d(kernel_size = 3 , stride = 2 , padding =1)
        
        self.Inception4a = Inception_block(480 ,192 ,96 ,208 ,16 ,48 ,64)
        self.Inception4b = Inception_block(512 ,160 ,112 ,224 ,24 ,64 ,64)
        self.Inception4c = Inception_block(512 ,128 ,128 ,256 ,24 ,64 ,64)
        self.Inception4d = Inception_block(512 ,112 ,114 ,288 ,32 ,64 ,64)
        self.Inception4e = Inception_block(528 ,256 ,160 ,320 ,32 ,128 ,128)
        
        self.maxpool4 = nn.MaxPool2d(kernel_size = 3 ,stride = 2 , padding =1)
        self.Inception5a = Inception_block(832 ,256 ,160 ,320 ,32 ,128 ,128)
        self.Inception5b = Inception_block(832 ,384 ,192 ,384 ,48 ,128 ,128)
        
        self.avgpool = nn.AvgPool2d(kernel_size = 7 , stride =  1)
        self.dropout  = nn.Dropout2d(p=0.4)
        self.fc1 = nn.Linear(1024 , num_classes)
    
    def forward(self ,x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        
        x = self.Inception3a(x)
        x = self.Inception3b(x)
        x = self.maxpool3(x)
        
        x = self.Inception4a(x)
        x = self.Inception4b(x)
        x = self.Inception4c(x)
        x = self.Inception4d(x)
        x = self.Inception4e(x)
        x = self.maxpool4(x)
        
        x = self.Inception5a(x)
        x = self.Inception5b(x)
        x = self.avgpool(x)
        
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc1(x)
        return x







        